## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv('/content/drive/MyDrive/software_marathon/ml_dl/news_data/train.csv')

Mounted at /content/drive


In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
###Drop Nan Values
df=df.dropna()


In [ ]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [ ]:
## Get the Dependent features
y=df['label']

In [ ]:
X.shape

(18285, 4)

In [ ]:
y.shape

(18285,)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.12.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [ ]:
messages=X.copy()

In [ ]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[170, 16, 3958, 1043, 752, 540, 3920, 4395, 1370, 2025],
 [2665, 1112, 1254, 3736, 3136, 1108, 9],
 [1905, 4286, 219, 3315],
 [1416, 3213, 1123, 2625, 351, 4586],
 [3387, 3136, 4178, 3521, 716, 2939, 3136, 3831, 1953, 636],
 [1928,
  531,
  2639,
  2129,
  4566,
  1696,
  455,
  4442,
  1804,
  3680,
  478,
  1266,
  4381,
  4474,
  9],
 [3288, 1657, 4474, 4811, 4327, 1832, 2949, 559, 4669, 958, 2474],
 [4356, 2968, 2593, 636, 766, 4586, 1696, 849, 4669, 958, 2474],
 [73, 462, 4873, 244, 4493, 1212, 3979, 555, 1696, 759],
 [3630, 4055, 891, 2930, 2824, 1261, 947, 1198],
 [728, 3612, 1632, 4448, 4539, 3876, 2857, 4078, 3449, 3333, 3384],
 [2625, 204, 752, 1212, 1696, 766],
 [2595, 918, 2646, 1113, 1990, 3287, 2204, 899, 1865],
 [3172, 83, 4112, 4098, 2472, 3553, 4598, 4669, 958, 2474],
 [3207, 2234, 3187, 2602, 4388, 4669, 958, 2474],
 [119, 4774, 4145, 212, 3976, 972, 975, 687, 2053, 1979],
 [2174, 271, 1112],
 [4456, 3083, 318, 2651, 1696, 1600, 350, 9],
 [4742, 4808, 1254, 2956, 328

### Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4395 1370 2025]
 [   0    0    0 ... 3136 1108    9]
 [   0    0    0 ... 4286  219 3315]
 ...
 [   0    0    0 ... 4669  958 2474]
 [   0    0    0 ... 2319 2200 2458]
 [   0    0    0 ... 2789 4182   13]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  170,
         16, 3958, 1043,  752,  540, 3920, 4395, 1370, 2025], dtype=int32)

In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(18285, (18285,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 19s 83ms/step - loss: 0.3425 - accuracy: 0.8398 - val_loss: 0.2118 - val_accuracy: 0.9118
Epoch 2/10
192/192 [==============================] - 5s 24ms/step - loss: 0.1501 - accuracy: 0.9413 - val_loss: 0.1910 - val_accuracy: 0.9176
Epoch 3/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1097 - accuracy: 0.9584 - val_loss: 0.2229 - val_accuracy: 0.9196
Epoch 4/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0883 - accuracy: 0.9695 - val_loss: 0.2194 - val_accuracy: 0.9115
Epoch 5/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0661 - accuracy: 0.9766 - val_loss: 0.2412 - val_accuracy: 0.9145
Epoch 6/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0552 - accuracy: 0.9811 - val_loss: 0.3014 - val_accuracy: 0.9153
Epoch 7/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0421 - accuracy: 0.9855 - val_loss: 0.2906 - val_accuracy: 0.907

### Performance Metrics And Accuracy

In [ ]:
y_pred=model.predict(X_test).round()
print(y)

189/189 [==============================] - 1s 3ms/step
0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[3086,  333],
       [ 201, 2415]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9115161557580779